# Base de datos Cassandra

## Conexion y borrado de datos

In [34]:
%load_ext cql

The cql extension is already loaded. To reload it, use:
  %reload_ext cql


In [35]:
%%cql
DROP KEYSPACE practica_8anu;

'No results.'

In [36]:
%%cql
CREATE KEYSPACE practica_8anu 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [37]:
%cql USE practica_8anu;

'No results.'

## Creacion de tablas

In [38]:
%%cql 
CREATE TABLE escaladores (
    id_escalador    int,
    nombre          text,
    sexo            text,
    pais            text,
    anio_comienzo   int,
    PRIMARY KEY (id_escalador)
);

'No results.'

In [39]:
%%cql 
CREATE TABLE ascensos_acumulados_hombres (
    id_escalador    int,
    num_ascensos    counter,
    PRIMARY KEY (id_escalador)
);

'No results.'

In [40]:
%%cql 
CREATE TABLE ascensos_acumulados_mujeres (
    id_escalador    int,
    num_ascensos    counter,
    PRIMARY KEY (id_escalador)
);

'No results.'

In [41]:
%%cql 
CREATE TABLE ascensos (
    uid                  uuid,
    id_escalador         int,
    nombre_escalador     text,
    pais_escalador       text,
    anio_comienzo        int,
    nombre_via           text,
    id_dificultad        int,
    grado_frances        text,
    tipo_encadenamiento  text,
    risco                text,
    pais_risco           text,
    PRIMARY KEY ((id_escalador), id_dificultad, uid)
)
WITH CLUSTERING ORDER BY (id_dificultad DESC);

'No results.'

In [42]:
%%cql
CREATE MATERIALIZED VIEW vm_ascensos_by_pais_risco AS
   SELECT * FROM ascensos
   WHERE id_escalador IS NOT NULL
         and uid IS NOT NULL
         and nombre_escalador IS NOT NULL
         and pais_escalador IS NOT NULL
         and anio_comienzo IS NOT NULL
         and nombre_via IS NOT NULL
         and id_dificultad IS NOT NULL
         and grado_frances IS NOT NULL
         and tipo_encadenamiento IS NOT NULL
         and risco IS NOT NULL
         and pais_risco IS NOT NULL
   PRIMARY KEY (pais_risco, id_dificultad, uid, id_escalador)

'No results.'

## Lectura de la informacion en Pandas

In [43]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];

## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver

In [44]:
df_ascensos_desnormalizado = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');
df_ascensos_desnormalizado = pd.merge(df_ascensos_desnormalizado, df_dificultades, on = ['id_dificultad'], how = 'inner');

# Renombrar campos por claridad
df_ascensos_desnormalizado = df_ascensos_desnormalizado.rename(
    columns={
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});


## Carga de datos en Cassandra

In [45]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('practica_8anu')

**CARGA EN TABLAS DE ASCENSOS Y ACUMULADOS**

In [46]:
import dateutil

def insert_ascensos(df):
    sql_insert = """
        INSERT INTO ascensos (
            uid,
            id_escalador,
            nombre_escalador,
            pais_escalador,
            anio_comienzo,
            nombre_via,
            id_dificultad,
            grado_frances,
            tipo_encadenamiento,
            risco,
            pais_risco
        ) VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    for index in df.index:
        id_escalador = df.ix[index, "id_escalador"]
        data = [
            id_escalador,
            df.ix[index, "nombre_escalador"],
            df.ix[index, "pais_escalador"],
            df.ix[index, "anio_comienzo"],
            df.ix[index, "nombre_via"],
            df.ix[index, "id_dificultad"],
            df.ix[index, "grado_frances"],
            df.ix[index, "tipo_encadenamiento"],
            df.ix[index, "risco"],
            df.ix[index, "pais_risco"]
        ]
        session.execute(sql_insert, data)
        
        if (df.ix[index, "sexo"] == 'Hombre'):
            session.execute("UPDATE ascensos_acumulados_hombres SET num_ascensos = num_ascensos + 1 WHERE id_escalador = %s", 
                            [id_escalador])
        else:
            session.execute("UPDATE ascensos_acumulados_mujeres SET num_ascensos = num_ascensos + 1 WHERE id_escalador = %s", 
                            [id_escalador])

In [47]:
insert_ascensos(df_ascensos_desnormalizado)

** CARGA EN TABLA DE ESCALADORES **

In [48]:
def insert_escaladores(df):
    sql_insert = """
        INSERT INTO escaladores (
            id_escalador,
            nombre,
            sexo,
            pais,
            anio_comienzo
        ) VALUES (%s, %s, %s, %s, %s)"""
    
    for index in df.index:
        data = [
            df.ix[index, "id_escalador"],
            df.ix[index, "nombre"],
            df.ix[index, "sexo"],
            df.ix[index, "pais"],
            df.ix[index, "comienzo"]]
        session.execute(sql_insert, data)

In [49]:
insert_escaladores(df_escaladores)

# RESPUESTAS A LAS PREGUNTAS

In [50]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

### 1.a) Los 10 escaladores (hombres) más activos (orden auxiliar por Id)

In [51]:
sql = """
SELECT id_escalador, num_ascensos
FROM ascensos_acumulados_hombres
"""
df = execute_query(sql)

df.sort_values('num_ascensos', ascending = False).head(10)

,id_escalador,num_ascensos
3616,50884,46
2919,20095,26
3886,20384,23
3663,66250,22
860,62639,22
3614,32034,20
3316,42086,20
3097,16672,20
1239,37830,19
695,66466,19


### 1.b) Los 10 escaladoras (mujeres) más activas (orden auxiliar por Id)

### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de via ascendete)

### 3. Dificultad media de los ascensos del escalador más activo

### 4.a) Los 10 ascensos mas dificiles

### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente